李萌萌的电子骨灰盒
----

这是一个通过ChatGLM模型训练的李萌萌的数字分身，你可以在问题栏目填入内容，或者什么都不填，来观察李萌萌到底会说些什么。
T4级别的GPU已经可以很胜任这个任务了。

In [ ]:
! nvidia-smi

### 安装依赖

In [ ]:
! git clone https://github.com/ljsabc/Fujisaki
%cd Fujisaki

%pip install -q -r requirements.txt
%pip install gradio

In [ ]:
import torch
import sys

from transformers import AutoTokenizer, GenerationConfig, AutoModel

torch.set_default_tensor_type(torch.cuda.HalfTensor)
model = AutoModel.from_pretrained("THUDM/chatglm-6b", revision="658202d", trust_remote_code=True).half().cuda()
tokenizer = AutoTokenizer.from_pretrained("THUDM/chatglm-6b", trust_remote_code=True, revision="658202d")
setattr(model, "lm_head_raw", model.lm_head)

The huggingface repo has something more than the QKV lora. We checkout a version that is the closest to the current version. 

In [ ]:
from peft import get_peft_model, LoraConfig, TaskType, PeftModel
peft_path = 'ljsabc/Fujisaki_GLM'      # change it to your own
model = PeftModel.from_pretrained(
       model,
       peft_path,
       torch_dtype=torch.float16
    )

# We have to use full precision, as some tokens are >65535
torch.set_default_tensor_type(torch.cuda.FloatTensor)
model.eval()

In [ ]:
def evaluate(context, temperature, top_p, top_k):
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        top_k=top_k,
        #repetition_penalty=1.1,
        num_beams=1,
        do_sample=True,
    )
    with torch.no_grad():
        input_text = f"Context: {context}Answer: " 
        ids = tokenizer([input_text], return_tensors="pt")
        inputs = ids.to("cuda")
        out = model.generate(
            **inputs,
            max_length=224,
            generation_config=generation_config
        )
        out = out.tolist()[0]
        decoder_output = tokenizer.decode(out)
        out_text = decoder_output.split("Answer: ")[1]
        return out_text

In [ ]:
def evaluate_stream(msg, history, temperature, top_p):
    generation_config = GenerationConfig(
        temperature=temperature,
        top_p=top_p,
        #repetition_penalty=1.1,
        num_beams=1,
        do_sample=True,
    )

    history.append([msg, None])

    context = ""
    if len(history) > 4:
        history.pop(0)

    for j in range(len(history)):
        history[j][0] = history[j][0].replace("<br>", "")

    # concatenate context
    for h in history[:-1]:
        context += h[0] + "||" + h[1] + "||"

    context += history[-1][0]
    context = context.replace(r'<br>', '')

    # TODO: Avoid the tokens are too long.
    CUTOFF = 224
    while len(tokenizer.encode(context)) > CUTOFF:
        # save 15 token size for the answer
        context = context[15:]

    h = []
    print("History:", history)
    print("Context:", context)
    for response, h in model.stream_chat(tokenizer, context, h, max_length=CUTOFF, top_p=top_p, temperature=temperature):
        history[-1][1] = response
        yield history, ""

In [ ]:
history = [['你是谁','我是喵喵'], ['你住在哪里', "我不知道"]]
for h in evaluate_stream("你在干什么", history, 1.0, 0.9):
    print(h)

In [ ]:
import gradio as gr

title = """<h1 align="center">李萌萌（Alter Ego）</h1>
<h3 align="center">这是一个通过ChatGLM模型训练的李萌萌的数字分身，你可以与她聊天，或者直接在文本框按下Enter，来观察李萌萌到底会说些什么。</h3>"""

footer =  """<p align='center'>项目在<a href='https://github.com/ljsabc/Fujisaki' target='_blank'>GitHub</a>上托管，基于清华的<a href='https://huggingface.co/THUDM/chatglm-6b' target='_blank'>THUDM/chatglm-6b</a>项目。</p>
<p align='center'><em>"I'm... a boy." --Chihiro Fujisaki</em></p>"""

with gr.Blocks() as demo:
    gr.HTML(title)
    state = gr.State()
    with gr.Row():
        with gr.Column(scale=2):
            temp = gr.components.Slider(minimum=0, maximum=1.1, value=0.95, label="Temperature",
                info="温度参数，越高的温度生成的内容越丰富，但是有可能出现语法问题。")
            top_p = gr.components.Slider(minimum=0.5, maximum=1.0, value=0.99, label="Top-p",
                info="top-p参数，只输出前p>top-p的文字，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。")
            #code = gr.Textbox(label="temp_output", info="解码器输出")
            #top_k = gr.components.Slider(minimum=1, maximum=200, step=1, value=25, label="Top k",
            #    info="top-k参数，下一个输出的文字会从top-k个文字中进行选择，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。")
            
        with gr.Column(scale=3):
            chatbot = gr.Chatbot(label="聊天框", info="")
            msg = gr.Textbox(label="输入框", placeholder="最近过得怎么样？",
                info="输入你的内容，按[Enter]发送。也可以什么都不填写生成随机数据。")
            clear = gr.Button("清除聊天")

    msg.submit(evaluate_stream, [msg, chatbot, temp, top_p], [chatbot, msg])
    clear.click(lambda: None, None, chatbot, queue=False)
    gr.HTML(footer)

demo.queue()
demo.launch(debug=False)


In [ ]:
import gradio as gr
gr.Interface(
    fn=evaluate,
    inputs=[
        gr.components.Textbox(
            lines=2, label="问题", placeholder="最近过得怎么样？",
            info="可以在这里输入你的问题。也可以什么都不填写生成随机数据。"
        ),
        #gr.components.Textbox(lines=2, label="Input", placeholder="none"),
        gr.components.Slider(minimum=0, maximum=1.1, value=1.0, label="Temperature",
            info="温度参数，越高的温度生成的内容越丰富，但是有可能出现语法问题。"),
        gr.components.Slider(minimum=0.5, maximum=1.0, value=0.99, label="Top p",
            info="top-p参数，只输出前p>top-p的文字，建议不要修改。"),
        gr.components.Slider(minimum=1, maximum=200, step=1, value=25, label="Top k",
            info="top-k参数，下一个输出的文字会从top-k个文字中进行选择，越大生成的内容越丰富，但也可能出现语法问题。数字越小似乎上下文的衔接性越好。"),
    ],
    outputs=[
        gr.inputs.Textbox(
            lines=5,
            label="Output",
        )
    ],
    title="李萌萌（Alter Ego）",
    description="这是一个通过ChatGLM模型训练的李萌萌的数字分身，你可以在问题栏目填入内容，或者什么都不填，来观察李萌萌到底会说些什么。",
).launch()